In [1]:
import glob
import re
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import *
import time
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from scipy import sparse
from sklearn import metrics
import _pickle as cPickle
import pandas as pd
import matplotlib.pyplot as plt
import csv

/home/samird121/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [110]:
assertives = []
f = open('../bias_lexicon/assertives_hooper1975.csv')
reader = csv.reader(f)
for line in reader:
    assertives.append(line[0])
    
factives = []
f = open('../bias_lexicon/factives_hooper1975.csv')
reader = csv.reader(f)
for line in reader:
    factives.append(line[0])
    
hedges = []
f = open('../bias_lexicon/hedges_hyland2005.csv')
reader = csv.reader(f)
for line in reader:
    hedges.append(line[0])
    
implicatives = []
f = open('../bias_lexicon/implicatives_karttunen1971.csv')
reader = csv.reader(f)
for line in reader:
    implicatives.append(line[0])

report_verbs = []
f = open('../bias_lexicon/report_verbs.csv')
reader = csv.reader(f)
for line in reader:
    report_verbs.append(line[0])
    
all_bias_words = assertives + factives+hedges+implicatives+report_verbs

In [111]:
any(word in all_bias_words for word in 'hi on the whole accuse asccuse'.split())

True

In [113]:
def preprocess(sentence):
    #take out "\n", which snuck its way into every downloaded sentence
    sentence = re.sub('\n', '', sentence)
    #sentence = re.sub(r"[\W+_]", ' ', sentence)
    #sentence = re.sub(r"[^a-zA-Z0-9]", ' ', sentence)
    #limit sentence to 100 words
    if len(sentence.split(' ')) > 100:
        sentence = ' '.join(sentence.split(' ')[0:99])
    if len(sentence.split()) > 2:
        return sentence
    else:
        return None

In [115]:
#neutral, liberal, conservative
sentences_separate = [[], []]
folders = ['../../reddit/lib_posts/', '../../reddit/con_posts/']
article_lengths = []

for i in range(2):
    folder = folders[i]
    
    #max number of sentences in article
    article_length_limit = 10000
    
    count = 1
    for fname in glob.iglob(folder+'*.txt', recursive=True):
        
        article_length = 0
        for line in open(fname):
            for s in line.split('.'):
                if (count > limit or article_length > article_length_limit) and 1 == 2:
                    break
                else:
                    s = preprocess(s)
                    if s is not None:
                        sentences_separate[i].append(s)
                        count += 1
                        article_length += 1
        article_lengths.append(article_length)

In [116]:
lib, con = sentences_separate[0], sentences_separate[1]
print(len(lib))
print(len(con))

44811
129203


In [122]:
np.random.seed(0)
shuffle=np.random.permutation(np.arange(len(lib)))
con_short = list(np.array(con)[shuffle])
len(con_short)

44811

In [123]:
sentences = []
labels = []

for i in range(len(lib)):
    sentences.append(lib[i])
    labels.append('liberal')
    
for i in range(len(con_short)):
    sentences.append(con_short[i])
    labels.append('conservative')
    
#for i in range(len(neu)):
#    sentences.append(neu[i])
#    labels.append('neutral')

In [124]:
lens = [len(x) for x in sentences]
print(min(lens))
print(max(lens))

8
719


In [126]:
print(sentences[10000])
print(labels[10000])
print('\n')
print(sentences[70000])
print(labels[70000])

you find it woven through every sage piece from the old-school pundits of the times and the hip insider websites like vox
liberal


“to fully achieve the goals we all share, we must do all we can to support and advocate the president-elect’s vision to restore and rebuild america’s greatness,” says ed rollins, a veteran gop strategist and a point man with the new organization
conservative


In [127]:
len(sentences)

89622

## Split into training/test set, vectorize, and run LR model

In [9]:
#.58379 (c = .5, min_df=5, W+)
#.595509 (c = .5, min_df=2)
#no re.suba-z [re[rp: .587]]

In [128]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(sentences, labels, random_state=2)
print(len(train_sentences))
print(len(test_sentences))

67216
22406


In [84]:
train_labels[2432]

'conservative'

In [129]:
cv = CountVectorizer(min_df=5, ngram_range=(0,2))
cv_train_sentences = cv.fit_transform(train_sentences)
cv_test_sentences = cv.transform(test_sentences)
print(len(cv.get_feature_names()))

58526


In [130]:
for C in [.5]:
    start = time.time()
    lr_base = LogisticRegression(C = C)
    lr_base.fit(cv_train_sentences, train_labels)
    preds = lr_base.predict(cv_test_sentences)
    print(C)
    print(classification_report(test_labels, preds))
    print("\nf1=score: "+str(metrics.f1_score(test_labels, preds, average='weighted')))
    print((time.time()-start), 's')

0.5
              precision    recall  f1-score   support

conservative       0.62      0.63      0.63     11090
     liberal       0.63      0.62      0.63     11316

 avg / total       0.63      0.63      0.63     22406


f1=score: 0.627509455442
10.882154703140259 s


## Examine model weights, scores, etc.

In [131]:
cv_featurenames = cv.get_feature_names()



weightIndeces_con = np.argsort(lr_base.coef_[0])[-20:]
weightIndeces_lib = np.argsort(lr_base.coef_[0])[0:20]
#weightIndeces_neu = np.argsort(lr_base.coef_[2])[-20:]
#weightIndeces_neu_neg = np.argsort(lr_base.coef_[2])[0:20]

print('Top 10 features for liberal:')
for index in weightIndeces_con: 
    print(cv_featurenames[index])

print('\nTop 10 features for conservative:')
for index in weightIndeces_lib:
    print(cv_featurenames[index])
    
#print('\nTop 10 features for neutral:')
#for index in weightIndeces_neu: 
#    print(cv_featurenames[index])
    
#print('\nBottom 10 features for neutral:')
#for index in weightIndeces_neu_neg: 
#    print(cv_featurenames[index])

Top 10 features for liberal:
pornography
bundy
ward
falsely
00
cooling
marijuana
maher
tigerswan
alec
arendt
murphy
trending
sanders
irv
deace
kobach
stein
ailes
flint

Top 10 features for conservative:
ddt
conservatism
beck
authoritarians
tb
uniform
fillon
venezuela
patman
adams
entitlements
kasich
genetic
jacobson
merkel
southern
bernie sanders
cameron
mattis
malaria


In [133]:
pred_probs = lr_base.predict_log_proba(cv_test_sentences)

In [134]:
con_scores = [score[0] for score in pred_probs]
top_con_indeces = np.argsort(con_scores)[-5:]
print('Top scored conservative sentences:')
for index in top_con_indeces:
    print(test_sentences[index])
    print(con_scores[index])
    print('\n\n*********')

Top scored conservative sentences:
• 72 percent say they are “absolutely certain” they will vote, 16 percent already have voted, 4 percent say they will “probably” vote, 4 percent say the chances are “50/50
-0.000296777436219


*********
as usual, the delegates are bound for the first presidential ballot (so the hard delegate totals don’t change) – but they’re also only bound to the first presidential ballot, which means that cruz will have a say in everything else
-0.000214385356828


*********
at such a brokered convention, sullivan’s plan to help swing it for rubio even if rubio has fewer delegates than trump is to convince the delegates to back rubio on a second ballot—where they would be technically unbound—and thereby essentially take the nomination away from trump, its rightful winner if he has the most delegates
-0.000125922242061


*********
if nobody gets to 50 percent of the delegates, if nobody gets to 1,237, then there’s a floor fight and delegates in most of these states,

In [89]:
lib_scores = [score[1] for score in pred_probs]
top_lib_indeces = np.argsort(lib_scores)[-20:]
print('Top scored liberal sentences:')
for index in top_lib_indeces:
    print(test_sentences[index])
    print(lib_scores[index])
    print('\n\n*********')

Top scored liberal sentences:
trump ’ s - head politics offer an intense high
-4.04252944924e-05


*********
these have on their more with border view the the are particularly sensitive since the is legally forbidden from directly on and its authority to conduct warrantless on is up for renewal in congress later this year
-3.72103972962e-05


*********
in the middle east , will concede that remains in the sphere of influence , and will withdraw all support for rebel
-3.71362738712e-05


*********
pollution is doing serious damage to long - term growth ; a 2006 estimate from china ' s then - deputy chief of its environmental agency that it off " roughly 10 percent " of
-3.71362738712e-05


*********
( peter / the post ) and the more that debate on , the worse it is for trump
-3.69041914968e-05


*********
fortunately for both our national - defense and the ’ pocketbook , this is not the case
-3.39288743364e-05


*********
and she said , " but caroline ' t hug you and i did
-2.0609076986

## attempting to apply model to IBC data

In [90]:
ibc_frame=pd.read_csv('../full_ibc_sentences.csv')


In [91]:
ibc_frame.head()

,label,sentence
0,Liberal,Forcing middle-class workers to bear a greater...
1,Liberal,Because it would not be worthwhile to bring a ...
2,Liberal,"Indeed , Lind argues that high profits and hig..."
3,Liberal,"In fairness , it should be noted that he devot..."
4,Liberal,Psychological tactics are social control techn...


In [92]:
ibc_frame = ibc_frame[ibc_frame['label']!='Neutral']

In [135]:
ibc_frame.groupby('label').count()

,sentence
label,
Conservative,1701
Liberal,2025


In [136]:
ibc_labels = list(ibc_frame['label'])
ibc_labels = [label.lower() for label in ibc_labels]
ibc_sentences = list(ibc_frame['sentence'])
#ibc_sentences = [preprocess(sentence) for sentence in ibc_sentences]
cv_ibc_sentences = cv.transform(ibc_sentences)
print(len(ibc_labels))
print(len(ibc_sentences))

3726
3726


In [137]:
IBC_preds = lr_base.predict(cv_ibc_sentences)
print(classification_report(ibc_labels, IBC_preds))
print("\nf1=score: "+str(metrics.f1_score(ibc_labels, IBC_preds, average='weighted')))
print(sum(IBC_preds==ibc_labels)/len(ibc_labels))

              precision    recall  f1-score   support

conservative       0.51      0.50      0.50      1701
     liberal       0.58      0.59      0.59      2025

 avg / total       0.55      0.55      0.55      3726


f1=score: 0.548791352321
0.549114331723


In [138]:

IBC_proba = lr_base.predict_proba(cv_ibc_sentences)
IBC_weak_labels = ibc_frame
IBC_weak_labels['con_score'] = [x[0] for x in IBC_proba]
IBC_weak_labels['lib_score'] = [x[1] for x in IBC_proba]
#IBC_weak_labels['neu_score'] = [x[2] for x in IBC_proba]
IBC_weak_labels['prediction'] = IBC_preds
IBC_weak_labels.head()

,label,sentence,con_score,lib_score,prediction
0,Liberal,Forcing middle-class workers to bear a greater...,0.122431,0.877569,liberal
1,Liberal,Because it would not be worthwhile to bring a ...,0.434909,0.565091,liberal
2,Liberal,"Indeed , Lind argues that high profits and hig...",0.039255,0.960745,liberal
3,Liberal,"In fairness , it should be noted that he devot...",0.187321,0.812679,liberal
4,Liberal,Psychological tactics are social control techn...,0.320699,0.679301,liberal


In [139]:
IBC_weak_labels.shape

(3726, 5)

In [140]:
IBC_weak_labels.to_csv('../IBC_weak_labels_conlibonly_noprog.csv')

In [ ]:
f = open('IBC_weak_labels.txt', 'w')
for i in range(IBC_sentences):
    


### Below are my attempts to implement sentiment analysis in the predictive model - not working yet, so you can probably ignore all this for now.

In [79]:
sid = SentimentIntensityAnalyzer()
pscores = []
i = 1
for sentence in sentences:
    pscores.append(sid.polarity_scores(sentence)['compound'])

In [80]:
train_sentences, test_sentences, train_labels, test_labels, train_pscores, test_pscores = train_test_split(sentences, labels, pscores, random_state=2)
print(len(train_sentences))
print(len(test_sentences))

87952
29318


In [84]:
cv = CountVectorizer(min_df=5, ngram_range=(0,2))
cv_train_sentences = cv.fit_transform(train_sentences)
cv_test_sentences = cv.transform(test_sentences)
print(len(cv.get_feature_names()))

60793


In [85]:
train_pscore_sparse = sparse.csr_matrix(train_pscores)
train_pscore_int = sparse.csc_matrix.multiply(np.transpose(train_pscore_sparse), cv_train_sentences)
stacked_train_sentences = sparse.hstack([cv_train_sentences, np.transpose(train_pscore_sparse), train_pscore_int])

test_pscore_sparse = sparse.csr_matrix(test_pscores)
test_pscore_int = sparse.csc_matrix.multiply(np.transpose(test_pscore_sparse), cv_test_sentences)
stacked_test_sentences = sparse.hstack([cv_test_sentences, np.transpose(test_pscore_sparse), test_pscore_int])

In [87]:
for C in [1]:
    start = time.time()
    lr_pscore = LogisticRegression(C = C)
    lr_pscore.fit(stacked_train_sentences, train_labels)
    preds = lr_pscore.predict(stacked_test_sentences)
    print(C)
    print(classification_report(test_labels, preds))
    print("\nf1=score: "+str(metrics.f1_score(test_labels, preds, average='weighted')))
    print((time.time()-start)/60, 'min')

1
              precision    recall  f1-score   support

conservative       0.54      0.54      0.54      9767
     liberal       0.57      0.55      0.56      9955
     neutral       0.60      0.62      0.61      9596

 avg / total       0.57      0.57      0.57     29318


f1=score: 0.568834883968
2.563340691725413 min


In [38]:
pred_probs_pscore = lr_pscore.predict_log_proba(stacked_test_sentences)
lib_scores_pscore = [score[1] for score in pred_probs_pscore]
top_con_indeces = np.argsort(lib_scores_pscore)[-20:]
for index in top_con_indeces:
    print(test_sentences[index])
    print(lib_scores[index])
    print(pscores[index])
    print(labels[index])
    print('\n\n*********')

(photo by lane turner/the boston globe via getty images) newton, ma - march 1 president bill clinton greets people at the newton free library in newton, mass., during massachusetts primary voting on march 1, 2016.
-0.0381862752356
-0.1263
liberal


*********
(photo by lane turner/the boston globe via getty images) newton, ma - march 1 president bill clinton greets people at the newton free library in newton, mass., during massachusetts primary voting on march 1, 2016.
-0.0381862752356
0.25
liberal


*********
# we can now finally start adding statements to the table for name, _, __, rating, date, text in stmnt_list"statements" cur.execute("insert into stmnt (pid, aid, s_date, text) values (" + str(pid_tablename.replace(" ", " ")) + ", " + str(aid_tablerating) + ", '" + date.strftime('%y%m%d') + "', '" + text.replace("'", "''") + "')") cur.connection.commit() def main() conn  sql.connect(host'localhost', user  "root", passwdmy_pass, db"mysql", charset  "utf8") cur  conn.cursor() cur.exe

## test on IBC data

In [87]:
ibc_frame

,label,sentence
0,Liberal,Forcing middle-class workers to bear a greater...
1,Liberal,Because it would not be worthwhile to bring a ...
2,Liberal,"Indeed , Lind argues that high profits and hig..."
3,Liberal,"In fairness , it should be noted that he devot..."
4,Liberal,Psychological tactics are social control techn...
5,Liberal,The uncontrolled profit motive is destroying h...
6,Liberal,Organizations representing the religious right...
7,Liberal,A market based on greed and fear has tugged on...
8,Liberal,THE CONSERVATIVE MOVEMENT IS ROOTED IN A COHER...
9,Liberal,"By eliminating the private insurer , you could..."


In [142]:
articles_1=pd.read_csv('../../.kaggle/datasets/snapcrack/all-the-news/articles1.csv')
articles_2=pd.read_csv('../../.kaggle/datasets/snapcrack/all-the-news/articles2.csv')
articles_3=pd.read_csv('../../.kaggle/datasets/snapcrack/all-the-news/articles3.csv')

In [143]:
all_articles=pd.concat([articles_1,articles_2,articles_3],ignore_index=True)
all_articles.shape

(142570, 10)

In [144]:
all_articles.columns

Index(['Unnamed: 0', 'id', 'title', 'publication', 'author', 'date', 'year',
       'month', 'url', 'content'],
      dtype='object')

In [145]:
nltk.sent_tokenize(all_articles['content'][0])

['WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win.',
 'The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for   and   Americans, handing House Republicans a big victory on    issues.',
 'But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement.',
 'That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government.',
 'To stave off that outcome, Republicans could find themselves in the awkward position of appropriating huge sums to temporarily prop up the Obama health care law, angering conservative vote

In [146]:
n_articles = all_articles.shape[0]
print(n_articles)

142570


In [147]:
pd.DataFrame([[2, 3, 4]])

,0,1,2
0,2,3,4


In [148]:
all_articles.shape[0]

142570

In [149]:
pd.DataFrame([[sentence, pub, proba[0][0], proba[0][1], proba[0][2], pred[0]]], columns=['sentence', 'source', 'con_score', 'lib_score', 'neu_score', 'weak_label'])

NameError: name 'pub' is not defined

In [83]:
count = 0
for i in range(all_articles.shape[0]):
    
    article = nltk.sent_tokenize(all_articles['content'][i])
    pub = all_articles['publication'][i]
    for sentence in article:
        cv_sentence = cv.transform([preprocess(sentence)])
        pred = lr_base.predict(cv_sentence)
        proba = lr_base.predict_proba(cv_sentence)
        if count == 0:
            df = pd.DataFrame([[pub, proba[0][0], proba[0][1], pred[0]]], columns=['source', 'con_score', 'lib_score',  'weak_label'])
        else:
            newrow = pd.DataFrame([[pub, proba[0][0], proba[0][1], pred[0]]], columns=['source', 'con_score', 'lib_score', 'weak_label'])
            df = pd.concat([df, newrow])
        #with open('../../ATN_reddit_weak_labels.csv', 'a') as f:
        #    if count == 0:
        #        df.to_csv(f, header=True)
        #    else:
        #        df.to_csv(f, header=False)
        count += 1
        if count % 10000 == 0:
            print(count)
    if count > 200000:
        break

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000


In [100]:
df.groupby('weak_label')['source'].count()

weak_label
conservative    53737
liberal         46292
neutral         99974
Name: source, dtype: int64

In [77]:
40.5*all_articles.shape[0]/3611

1599.026585433398

In [151]:
atn_sentences = [sentence for sentence in all_articles['content']]
atn_preprocessed_sentences = [preprocess(sentence) for sentence in atn_sentences]
cv_atn_sentences = cv.transform(atn_sentences)

KeyboardInterrupt: 

In [29]:
atn_sentences[1]

'After the bullet shells get counted, the blood dries and the votive candles burn out, people peer down from   windows and see crime scenes gone cold: a band of yellow police tape blowing in the breeze. The South Bronx, just across the Harlem River from Manhattan and once shorthand for urban dysfunction, still suffers violence at levels long ago slashed in many other parts of New York City. And yet the city’s efforts to fight it remain splintered, underfunded and burdened by scandal. In the 40th Precinct, at the southern tip of the Bronx, as in other'

In [ ]:
ATN_preds = lr_base.predict(cv_atn_sentences)
ATN_proba = lr_base.predict_proba(cv_atn_sentences)

In [ ]:
ATN_weak_labels = ibc_frame
ATN_weak_labels['con_score'] = [x[0] for x in IBC_proba]
ATN_weak_labels['lib_score'] = [x[1] for x in IBC_proba]
ATN_weak_labels['neu_score'] = [x[2] for x in IBC_proba]
ATN_weak_labels['prediction'] = IBC_preds
ATN_weak_labels.head()

In [ ]:
atn_sentences = atn_sentences[atn_sentences['lib_score']!='lib_score']
atn_sentences

In [152]:
atn_selected = pd.read_csv('../all_the_news_sentences.csv')

In [153]:
atn_selected.head()

,Unnamed: 0,Sentence,Source
0,0,On Tuesday’s broadcast of CNN’s “Situation Roo...,Breitbart
1,1,"” Zeleny said, “I think a question that this ...",Breitbart
2,2,And she’s having a difficult time in federal p...,Breitbart
3,3,"But, to me, that is a central question here.",Breitbart
4,4,"Without that, you have to wonder if the outcom...",Breitbart


In [154]:
atn_selected.shape[0]

193744

In [156]:
count = 0

for i in range(atn_selected.shape[0]):
    pub = atn_selected['Source'][i]
    sentence = atn_selected['Sentence'][i]
    cv_sentence = cv.transform([sentence])
    pred = lr_base.predict(cv_sentence)
    proba = lr_base.predict_proba(cv_sentence)
    df = pd.DataFrame([[sentence, pub, proba[0][0], proba[0][1], pred[0]]], columns=['sentence', 'source', 'con_score', 'lib_score', 'weak_label'])

    with open('../ATN_selected_reddit_weak_labels_conlibonly_noprog.csv', 'a') as f:
        if count == 0:
            df.to_csv(f, header=True)
        else:
            df.to_csv(f, header=False)
    count += 1
    if count % 10000 == 0:
        print(count)
#if count > 200000:
#    break

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000


In [157]:
atn_df = pd.read_csv('../ATN_selected_reddit_weak_labels_conlibonly_noprog.csv')

/home/samird121/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [44]:
atn_df = atn_df[atn_df['weak_label'] != 'weak_label']

In [47]:
atn_df.groupby(['source', 'weak_label']).mean()

Unnamed: 0
source           weak_label              
Breitbart        conservative         0.0
                 liberal              0.0
Business Insider conservative         0.0
                 liberal              0.0
CNN              conservative         0.0
                 liberal              0.0
Guardian         conservative         0.0
                 liberal              0.0
National Review  conservative         0.0
                 liberal              0.0
Reuters          conservative         0.0
                 liberal              0.0

In [46]:
#df = atn_df.groupby(['source'])['con_score', 'lib_score'].mean()
#df.plot.bar()
#plt.ylabel('Average Score')
#plt.title('Average score for predicted category\n(predicted by weak annotator)')
#plt.show()

DataError: No numeric types to aggregate

In [95]:
np.array(lib)[indeces]

array(['they own a four-bedroom, 2',
       '"the cop on the street is always being second-guessed, always going to have 5 million eyes on him',
       'the new deal calls for service members who received payments under the santander settlement to receive $5,500, and for those who did not, to receive $11,000',
       ..., 'home is in a book',
       'fifty percent of the proceeds go to fund planned parenthood',
       '“an important element of the continued market growth is the development of the charging infrastructure,” said joseph g'],
      dtype='<U1198')

In [96]:
indeces =shuffle=np.random.permutation(np.arange(100000))
lib_shuffle = np.array(lib)[indeces]
indeces =np.random.choice(range(100000), 100000, replace=False)
con_shuffle = np.array(con)[indeces]

In [99]:
count = 0

for sentence in lib_shuffle:
    cv_sentence = cv.transform([sentence])
    pred = lr_base.predict(cv_sentence)
    proba = lr_base.predict_proba(cv_sentence)
    
    df = pd.DataFrame([[sentence, proba[0][0], proba[0][1], pred[0]]], columns=['sentence', 'con_score', 'lib_score', 'weak_label'])
    with open('../reddit_subs_weak_labels_conlibonly.csv', 'a') as f:
        if count == 0:
            df.to_csv(f, header=True)
        else:
            df.to_csv(f, header=False)
    count += 1
    if count % 10000 == 0: 
        print(count)
for sentence in con_shuffle:
    cv_sentence = cv.transform([sentence])
    pred = lr_base.predict(cv_sentence)
    proba = lr_base.predict_proba(cv_sentence)
    
    df = pd.DataFrame([[sentence, proba[0][0], proba[0][1], pred[0]]], columns=['sentence','con_score', 'lib_score', 'weak_label'])
    with open('../reddit_subs_weak_labels_conlibonly.csv', 'a') as f:
        if count == 0:
            df.to_csv(f, header=True)
        else:
            df.to_csv(f, header=False)
    count += 1
    if count % 10000 == 0:
        print(count)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000


In [ ]:
'We must respect tradition and god'
cv_sentence = cv.transform([sentence])
pred = lr_base.predict(cv_sentence)
proba = lr_base.predict_proba(cv_sentence)

In [141]:
count = 0

for i in range(len(sentences)):
    
    sentence = sentences[i]
    label = labels[i]
    df = pd.DataFrame([[sentence,label]], columns=['sentence','weak_label'])
    with open('../  ', 'a') as f:
        if count == 0:
            df.to_csv(f, header=True)
        else:
            df.to_csv(f, header=False)
    count += 1
    if count % 10000 == 0: 
        print(count)


10000
20000
30000
40000
50000
60000
70000
80000
